# Thesis Results (Upright Balancing)

This notebook is the **polished** analysis entrypoint for thesis plots/tables.

## Workflow
1. Run experiments with logging (see `tools/experiment_quickstart.md`).
2. Generate a report folder:
   - `./.venv/bin/python tools/analyze_experiments.py --glob 'logs/balance/session_202602*'`
3. Open this notebook and point it at the generated report directory.

Notes:
- CSV is ~50 Hz; any spectrum plots are **low-frequency only** (≤25 Hz).
- Event annotations rely on robust `[STATUS]` matching. If alignment is unreliable, the CLI warns and skips time-precise annotations.


In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

# Pick the latest report by default
reports_root = Path("../reports/thesis")
report_dirs = [p for p in reports_root.glob("*") if p.is_dir()]
if not report_dirs:
    raise FileNotFoundError("No reports found under ../reports/thesis. Run tools/analyze_experiments.py first.")

report_dir = max(report_dirs, key=lambda p: p.stat().st_mtime)
report_dir

In [ ]:
trials = pd.read_csv(report_dir / "metrics_trials.csv")
trials.head()

In [ ]:
# Quick sanity summary
cols = [
    "duration_s",
    "alpha_rms_deg",
    "alpha_max_abs_deg",
    "theta_drift_slope_deg_s",
    "theta_max_abs_deg",
    "acc_rms_steps_s2",
    "acc_max_abs_steps_s2",
    "sat_pct",
    "clamped_pct",
]
existing = [c for c in cols if c in trials.columns]
trials.groupby("mode")[existing].agg(["mean", "median", "std", "count"])

In [ ]:
# Distribution plots (anti-cherry-picking)
fig, axes = plt.subplots(1, 2, figsize=(12, 4.5))
trials.boxplot(column="alpha_rms_deg", by="mode", ax=axes[0])
axes[0].set_title("α RMS by mode")
axes[0].set_ylabel("deg")
axes[0].grid(True, alpha=0.25)

trials.boxplot(column="theta_drift_slope_deg_s", by="mode", ax=axes[1])
axes[1].set_title("θ drift slope by mode")
axes[1].set_ylabel("deg/s")
axes[1].grid(True, alpha=0.25)

fig.suptitle("")
fig.tight_layout()
plt.show()

In [ ]:
# Nudge metrics (if present)
nudge_path = report_dir / "metrics_nudge_steps.csv"
if nudge_path.exists():
    nudge = pd.read_csv(nudge_path)
    display(nudge.head())
    display(nudge.groupby("mode")[["rise_time_s", "overshoot_deg", "theta_ss_err_deg", "alpha_max_abs_deg"]].agg(["mean", "median", "std", "count"]))
else:
    print("No nudge metrics file found.")